## Домашнє завдання №5
## Класифікування активності за даними акселерометра

### 1. Отримання датасету

In [131]:
mypath = '.\data'

In [132]:
from os import listdir
from os.path import isfile, join
import numpy as np
import pandas as pd

In [133]:
# отримаємо список підкаталогів в теці "data"
activites = listdir(mypath)
activites

['idle', 'running', 'stairs', 'walking']

In [134]:
# Виведемо кількість файлів в кожній теці активності
for act in activites:
    path = join(mypath, act)  # створення шляху для активності
    frames = listdir(path)  # отримання списку файлів в 'path'
    print(act, ': ', len(frames))  # виведення кількості файлів в 'act'

idle :  1039
running :  3408
stairs :  165
walking :  1850


In [135]:
# Виведемо для прикладу зміст шостого файлу з теки 'walking' ('сирі' дані)
frames = listdir(join(mypath, 'walking'))
frame = pd.read_csv(join(join(mypath, 'walking'),frames[5]))
frame  # датасет містить посекундні дані переміщення, кожний рядок - одна секунда

,accelerometer_X,accelerometer_Y,accelerometer_Z
0,-4.352658,-9.327810,0.713472
1,-3.955221,-16.496050,3.438074
2,-2.705448,-7.436390,1.383849
3,-4.577713,-17.645266,1.944092
4,7.192182,-25.201366,-16.903065
5,1.575385,-10.620679,1.589750
6,-0.028730,-5.425261,0.823605
7,3.418920,-6.675034,1.158794
8,1.891419,-11.669339,-1.570596
9,-1.647211,-5.305551,9.169792


## 2. Підготовка даних

In [136]:
# Імпортуємо модуль для оцінювання середньоквадратичної помилки "mean_squared_error" та
# модуль для обчислення середньої абсолютної помилки "mean_absolute_error".
# Ці метрики використовують для вимірювання різниці між прогнозованими та фактичними значеннями.
from sklearn.metrics import mean_squared_error, mean_absolute_error
# Імпортуємо модуль для перетворення Фур'є, тобто він перетворює дані з часової області в частотну.
# "fft" - для комплексних даних (містять дійсну та уявну частину), "rfft" - для дійсних
from scipy.fftpack import fft, rfft
# Імпортуємо модуль для первинної обробки даних
from sklearn import preprocessing

In [137]:
# Створимо список часових ознак для датасету (з лекційного матеріалу)
features = [
    'skew_X',  # асиметрія розподілу даних
    'skew_Y',
    'skew_Z',
    'kurtosis_X',  # гострота пику розподілу даних
    'kurtosis_Y',
    'kurtosis_Z',
    'max_X',
    'max_Y',
    'max_Z',
    'min_X',
    'min_Y',
    'min_Z',
    'mean_X',
    'mean_Y',
    'mean_Z',
    'std_X',  # стандартне відхилення, що являється мірою розкиду даних
    'std_Y',
    'std_Z',
    'variance_X',  # дисперсія (розкид даних)
    'variance_Y',
    'variance_Z',
    'median_X',
    'median_Y',
    'median_Z',
    'index_max_X',
    'index_max_Y',
    'index_max_Z',
    'index_min_X',
    'index_min_Y',
    'index_min_Z',
    'corelation_X_Y',
    'corelation_X_Z',
    'corelation_Y_Z',
    'mae_X',  # середня абсолютна різниця між значеннями даних та прогнозними значеннями
    'mae_Y',
    'mae_Z',
    'rmse_X',  # середньоквадратична різниця між значеннями даних та прогнозними значеннями
    'rmse_Y',
    'rmse_Z'
    ]

len(features)

39

In [138]:
# Створимо ф-ю, яка рахуватиме статистичні ознаки
def get_stat_features(frame):

    features = []
    features = np.array(features)  # перетворення порожнього списку в массив NumPy
    features = np.concatenate((features, frame.skew(axis=0).values), axis=0)
    # конкатенація масиву 'features' з масивом ознак, обчислених з використанням методу 'skew' (асиметрія) для DataFrame 'frame'
    # 'axis=0' - конкатенація виконується по стовпчикам (по вертикалі)
    # '.values' - перетворює результати в массив NumPy
    # 'np.concatenate' - додавання стовпців до масиву
    features = np.concatenate((features, frame.kurt(axis=0).values), axis=0)
    # '.kurt' - kurtosis, розрахунок ексцеса (гострота пику розподілу даних)
    features = np.concatenate((features, frame.max(axis=0).values), axis=0)
    features = np.concatenate((features, frame.min(axis=0).values), axis=0)
    features = np.concatenate((features, frame.mean(axis=0).values), axis=0)
    features = np.concatenate((features, frame.std(axis=0).values), axis=0)
    features = np.concatenate((features, frame.var(axis=0).values), axis=0)
    features = np.concatenate((features, frame.median(axis=0).values), axis=0)
    features = np.concatenate((features, frame.idxmax(axis=0).values), axis=0)
    features = np.concatenate((features, frame.idxmin(axis=0).values), axis=0)

    # створення матриці кореляції для всіх пар колонок
    correlations = frame.corr()  # розрахунок коефіцієнтів кореляції для кожної пари колонок
    # створення матриці з конкретними значеннями кореляції (на базі матриці "correlations")
    corr = np.array([
        correlations['accelerometer_X']['accelerometer_Y'],
        correlations['accelerometer_X']['accelerometer_Z'],
        correlations['accelerometer_Y']['accelerometer_Z']
    ])

    # додамо стовпчики з коефіцієнтами кореляції до матриці features
    features = np.concatenate((features, corr), axis=0)

    # створення нових колонок у "frame"
    frame['mean_X'] = frame.mean(axis=0)['accelerometer_X']  # міститиме середнє знач. для даних в колонці 'accelerometer_X'
    frame['mean_Y'] = frame.mean(axis=0)['accelerometer_Y']
    frame['mean_Z'] = frame.mean(axis=0)['accelerometer_Z']
    
    # розрахуємо середню абсолютну помилку (MAE) між двома стовпчиками
    mae_X = mean_absolute_error(frame['accelerometer_X'], frame['mean_X'])
    mae_Y = mean_absolute_error(frame['accelerometer_Y'], frame['mean_Y'])
    mae_Z = mean_absolute_error(frame['accelerometer_Z'], frame['mean_Z'])

    # розрахуємо корінь середньоквадратичної помилки (RMSE) між двома стовпчиками
    rmse_x = np.sqrt(mean_squared_error(frame['accelerometer_X'], frame['mean_X']))
    rmse_y = np.sqrt(mean_squared_error(frame['accelerometer_Y'], frame['mean_Y']))
    rmse_z = np.sqrt(mean_squared_error(frame['accelerometer_Z'], frame['mean_Z']))

    # створення масиву
    metrics = np.array([mae_X, mae_Y, mae_Z, rmse_x, rmse_y, rmse_z])
    # поєднання масивів
    features = np.concatenate((features, metrics), axis=0)

    return features

In [139]:
len(get_stat_features(frame))

39

In [140]:
get_stat_features(frame)

array([-3.23847418e-01,  1.62237440e-01,  2.06164601e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  1.34180485e-01,  5.42626937e-01,
        1.03261163e+01,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        1.11138840e+01,  6.43561400e+00,  3.25994500e+01,  2.85548467e-01,
       -1.09161231e+01,  3.66472567e-01, -1.22774670e+01, -2.52013660e+01,
       -1.69030650e+01,  2.85548467e-01, -1.09161231e+01,  3.66472567e-01,
        2.85548467e-01, -1.09161231e+01,  3.66472567e-01,  2.85548467e-01,
       -1.09161231e+01,  3.66472567e-01,  5.09090559e+00,  6.82046510e+00,
        7.81198587e+00,  5.64601285e-17,  3.61344823e-15,  1.12920257e-16,
        2.59173197e+01,  4.65187441e+01,  6.10271232e+01,  3.18774611e-33,
        1.30570081e-29,  1.27509845e-32,  7.73327500e-01, -1.03453455e+01,
        7.68538500e-01,  2.85548467e-01, -1.09161231e+01,  3.66472567e-01,
        1.80000000e+01,  2.50000000e+01,  1.10000000e+01,  0.00000000e+00,
        0.00000000e+00,  

In [141]:
# Створимо ф-ю, яка рахуватиме коефіцієнти швидкого перетворення Фур'є та нормалізуватиме їх
def get_fft_features(frame):
    
    fft_X = np.array(preprocessing.normalize([rfft(frame['accelerometer_X'].values)])[0])
    fft_Y = np.array(preprocessing.normalize([rfft(frame['accelerometer_Y'].values)])[0])
    fft_Z = np.array(preprocessing.normalize([rfft(frame['accelerometer_Z'].values)])[0])

    features = np.concatenate((fft_X, fft_Y), axis=0)
    features = np.concatenate((features, fft_Z), axis=0)

    return features

In [142]:
len(get_fft_features(frame))

90

In [143]:
get_fft_features(frame)

array([ 0.07921776, -0.36261872, -0.23872122, -0.00875992, -0.17069203,
        0.29059423,  0.11034306, -0.27884912,  0.4654557 , -0.02147603,
        0.06779929, -0.11895174,  0.1659198 , -0.16414777, -0.06431846,
        0.12806833, -0.12678634, -0.03159774, -0.23551801,  0.08439857,
       -0.09456619, -0.15296785,  0.25380562, -0.05073099, -0.02089613,
        0.04253843,  0.18901115, -0.1205775 , -0.00540667,  0.24340973,
       -0.91415542, -0.04135142, -0.01454753, -0.04903838,  0.10016657,
        0.01045201, -0.03044975,  0.15831376,  0.01927306,  0.04191766,
       -0.00606573,  0.09558249, -0.06538515, -0.04711361,  0.08575738,
       -0.04597333,  0.12892033, -0.15039368,  0.09959709, -0.06760828,
        0.02444813,  0.07059414, -0.07885837,  0.04744685, -0.00142663,
        0.06534237,  0.0965083 ,  0.03595168, -0.0256773 , -0.11523349,
        0.0651826 , -0.10328124, -0.19088715, -0.03101319,  0.28042941,
        0.21904609, -0.03904686,  0.01179265, -0.25602528,  0.12

#### Перевірка працездатності функціоналу ф-ї get_stat_features (чернетка)

In [144]:
frame.skew(axis=0)  # розрахунок асиметрії даних для кожного стовпця 'frame'

accelerometer_X   -0.323847
accelerometer_Y    0.162237
accelerometer_Z    2.061646
mean_X             0.000000
mean_Y             0.000000
mean_Z             0.000000
dtype: float64

In [145]:
frame.kurt(axis=0)  # розрахунок ексцеса для кожного стовпця 'frame'

accelerometer_X     0.134180
accelerometer_Y     0.542627
accelerometer_Z    10.326116
mean_X              0.000000
mean_Y              0.000000
mean_Z              0.000000
dtype: float64

In [146]:
frame.max(axis=0)  # розрахунок максимального значення для кожного стовпця 'frame'

accelerometer_X    11.113884
accelerometer_Y     6.435614
accelerometer_Z    32.599450
mean_X              0.285548
mean_Y            -10.916123
mean_Z              0.366473
dtype: float64

In [147]:
frame.min(axis=0)  # розрахунок мінімального значення для кожного стовпця 'frame'

accelerometer_X   -12.277467
accelerometer_Y   -25.201366
accelerometer_Z   -16.903065
mean_X              0.285548
mean_Y            -10.916123
mean_Z              0.366473
dtype: float64

In [148]:
frame.mean(axis=0)  # розрахунок середнього значення для кожного стовпця 'frame'

accelerometer_X     0.285548
accelerometer_Y   -10.916123
accelerometer_Z     0.366473
mean_X              0.285548
mean_Y            -10.916123
mean_Z              0.366473
dtype: float64

In [149]:
frame.std(axis=0)  # розрахунок стандартного відхилення для кожного стовпця 'frame'

accelerometer_X    5.090906e+00
accelerometer_Y    6.820465e+00
accelerometer_Z    7.811986e+00
mean_X             5.646013e-17
mean_Y             3.613448e-15
mean_Z             1.129203e-16
dtype: float64

In [150]:
frame.var(axis=0)  # розрахунок дисперсії (розкид даних) для кожного стовпця 'frame'

accelerometer_X    2.591732e+01
accelerometer_Y    4.651874e+01
accelerometer_Z    6.102712e+01
mean_X             3.187746e-33
mean_Y             1.305701e-29
mean_Z             1.275098e-32
dtype: float64

In [151]:
frame.idxmax(axis=0)  #  індекс першого входження максимального значення для кожного стовпця 'frame'

accelerometer_X    18
accelerometer_Y    25
accelerometer_Z    11
mean_X              0
mean_Y              0
mean_Z              0
dtype: int64

In [152]:
frame.idxmin(axis=0)  #  індекс першого входження мінімального значення для кожного стовпця 'frame'

accelerometer_X    25
accelerometer_Y     4
accelerometer_Z     4
mean_X              0
mean_Y              0
mean_Z              0
dtype: int64

#### Функції для створення масивів NumPy з різними даними

In [154]:
# Створимо другу ф-ю, яка виконуватиме підготовку даних
def class_data_stat_fft_prepare(class_name, class_number):
    """
    Створення двох масивів NumPy: 
    Х - з розрахованими на основі датасету часовими ознаками та нормалізованими коефіцієнтами швидкого перетворення Фур'є;
    у - з мітками ("class_number")
    """

    path = join(mypath, class_name)  # формування шляху до директорії певного класу даних
    X = []
    for item in listdir(path):  # ітерація по файлам в "path"
        frame = pd.read_csv(join(path, item)) # кожний файл зчитується
        features_stat = get_stat_features(frame)  # розрахунок всіх часових ознак
        features_fft = get_fft_features(frame)  # розрахунок коефіцієнтів швидкого перетворення Фур'є та їх нормалізація
        features = np.concatenate((features_stat, features_fft), axis=0)
        X.append(features)

    y = [class_number]*len(X)  # створення списку однакових значень "class_number" довжиною "len(X)"
    X = np.array(X)  # перетворення списку в масив NumPy
    y = np.array(y)  # перетворення списку в масив NumPy
    return X, y

In [155]:
# Створимо третю ф-ю, яка виконуватиме підготовку даних
def class_data_fft_prepare(class_name, class_number):
    """
    Створення двох масивів NumPy: 
    Х - з розрахованими на основі датасету нормалізованими коефіцієнтами швидкого перетворення Фур'є;
    у - з мітками ("class_number")
    """
    
    path = join(mypath, class_name)  # формування шляху до директорії певного класу даних
    X = []
    for item in listdir(path):  # ітерація по файлам в "path"
        frame = pd.read_csv(join(path, item)) # кожний файл зчитується
        features = get_fft_features(frame)  # розрахунок коефіцієнтів швидкого перетворення Фур'є та їх нормалізація
        X.append(features)

    y = [class_number]*len(X)  # створення списку однакових значень "class_number" довжиною "len(X)"
    X = np.array(X)  # перетворення списку в масив NumPy
    y = np.array(y)  # перетворення списку в масив NumPy
    return X, y

In [156]:
# Створимо четверту ф-ю, яка виконуватиме підготовку даних
def class_data_stat_prepare(class_name, class_number):
    """
    Створення двох масивів NumPy: 
    Х - з розрахованими на основі датасету зі статистичними ознаками;
    у - з мітками ("class_number")
    """
    
    path = join(mypath, class_name)  # формування шляху до директорії певного класу даних
    X = []
    for item in listdir(path):  # ітерація по файлам в "path"
        frame = pd.read_csv(join(path, item)) # кожний файл зчитується
        features = get_stat_features(frame)  # розрахунок статистичних ознак
        X.append(features)

    y = [class_number]*len(X)  # створення списку однакових значень "class_number" довжиною "len(X)"
    X = np.array(X)  # перетворення списку в масив NumPy
    y = np.array(y)  # перетворення списку в масив NumPy
    return X, y

#### Створення датасету

In [157]:
# Створимо ф-ю, яка створюватиме масив даних та масив міток (номерів класів даних - від 0 до 3), використовуючи одну з 4х вище написаних ф-й
def create_dataset(class_prepare):

    X_idle, y_idle = class_prepare('idle', 0)
    X_walking, y_walking = class_prepare('walking', 1)
    X_stairs, y_stairs = class_prepare('stairs', 2)
    X_running, y_running = class_prepare('running', 3)

    X = np.concatenate((X_idle, X_walking), axis=0)
    X = np.concatenate((X, X_stairs), axis=0)
    X = np.concatenate((X, X_running), axis=0)

    Y = np.concatenate((y_idle, y_walking))
    Y = np.concatenate((Y, y_stairs))
    Y = np.concatenate((Y, y_running))

    return X, Y

In [158]:
# Створимо масив даних на основі статистичних ознак та з номерними мітками
X_stat, Y_stat = create_dataset(class_data_stat_prepare)

# Запишемо отримані дані у файли в бінарному форматі
import pickle
with open('stat_data_X.pickle', 'wb') as f:
    pickle.dump(X_stat, f)

with open('stat_data_Y.pickle', 'wb') as f:
    pickle.dump(Y_stat, f)

In [159]:
X_stat

array([[ 4.04430764,  3.64747035, -2.36531183, ...,  1.01894178,
         1.13814657,  0.40006418],
       [ 2.30067561,  0.56822274, -2.40992225, ...,  0.12373242,
         0.04725101,  0.02499156],
       [ 0.60425841, -1.27292011, -0.09747658, ...,  0.01537931,
         0.01610751,  0.01428421],
       ...,
       [-0.11333213,  0.11486419,  0.6462055 , ...,  9.17270206,
        11.42746342,  7.20864106],
       [ 0.15870895,  0.20634991,  1.59898321, ...,  8.31015509,
        11.28133316,  9.34949964],
       [ 0.15289451,  0.20889288,  1.48055216, ...,  8.32459024,
        11.16749279,  9.48298175]])

In [160]:
Y_stat

array([0, 0, 0, ..., 3, 3, 3])

In [161]:
# Створимо масив даних на основі перетворення Фур'є та з номерними мітками
X_fft, Y_fft = create_dataset(class_data_fft_prepare)

# Запишемо отримані дані у файли в бінарному форматі
with open('fft_data_X.pickle', 'wb') as f:
    pickle.dump(X_fft, f)

with open('fft_data_Y.pickle', 'wb') as f:
    pickle.dump(Y_fft, f)

In [162]:
X_fft

array([[ 2.38227099e-01,  2.97848393e-01, -1.77686579e-01, ...,
         7.48720238e-04,  1.76652916e-03,  9.13509330e-04],
       [-7.44333032e-01,  6.58117188e-02, -7.76931362e-02, ...,
         1.17151312e-04, -4.27179315e-04, -3.26697109e-04],
       [-9.93968876e-01, -1.48712954e-02,  5.41564486e-03, ...,
         1.89190319e-04, -3.25180079e-04,  8.16979087e-05],
       ...,
       [ 4.94501867e-01, -2.19979802e-01, -4.19612638e-02, ...,
         1.09872833e-01, -4.99230168e-02, -4.25208043e-01],
       [ 5.89314956e-01, -9.84167445e-02, -1.11841867e-01, ...,
        -5.52941061e-02, -4.35714065e-02, -4.42128223e-01],
       [ 5.91829128e-01, -4.71910009e-02, -1.36855263e-01, ...,
        -3.06142176e-02, -3.21755312e-02, -3.95902853e-01]])

In [163]:
Y_fft

array([0, 0, 0, ..., 3, 3, 3])

In [164]:
# Створимо масив даних на основі часових ознак та номерних міток
X_fft_stat, Y_fft_stat = create_dataset(class_data_stat_fft_prepare)

# Запишемо отримані дані у файли в бінарному форматі
with open('fft_stat_data_X.pickle', 'wb') as f:
    pickle.dump(X_fft_stat, f)

with open('fft_stat_data_Y.pickle', 'wb') as f:
    pickle.dump(Y_fft_stat, f)

In [165]:
X_fft_stat

array([[ 4.04430764e+00,  3.64747035e+00, -2.36531183e+00, ...,
         7.48720238e-04,  1.76652916e-03,  9.13509330e-04],
       [ 2.30067561e+00,  5.68222742e-01, -2.40992225e+00, ...,
         1.17151312e-04, -4.27179315e-04, -3.26697109e-04],
       [ 6.04258414e-01, -1.27292011e+00, -9.74765802e-02, ...,
         1.89190319e-04, -3.25180079e-04,  8.16979087e-05],
       ...,
       [-1.13332133e-01,  1.14864190e-01,  6.46205498e-01, ...,
         1.09872833e-01, -4.99230168e-02, -4.25208043e-01],
       [ 1.58708946e-01,  2.06349906e-01,  1.59898321e+00, ...,
        -5.52941061e-02, -4.35714065e-02, -4.42128223e-01],
       [ 1.52894505e-01,  2.08892875e-01,  1.48055216e+00, ...,
        -3.06142176e-02, -3.21755312e-02, -3.95902853e-01]])

In [166]:
Y_fft_stat

array([0, 0, 0, ..., 3, 3, 3])

#### Завантаження датасету

In [167]:
# Ф-я завантаження датасету. Приймає аргумент: 'fft', 'stat' або 'fft_stat'
def open_data_set(flag):
    name = flag + '_data_'
    
    with open(name + 'X.pickle', 'rb') as f:
        X = pickle.load(f)
    with open(name + 'Y.pickle', 'rb') as f:
        Y = pickle.load(f)
    return X, Y

In [168]:
# Перевіримо, що завантажений датасет дорівнює оригіналу
# Завантажимо датасет 'fft_data_Y.pickle'
X, Y = open_data_set('fft')
# Перевірка коректності роботи завантажувача pickle
np.array_equal(X, X_fft)  # перевірка однаковості

True

In [169]:
# Створимо ф-ю для підрахунку кількості індексів класів
def make_report(y_test_proba, y_test):

    idle_test = 0
    walking_test = 0
    stairs_test = 0
    running_test = 0

    for i in y_test_proba:
        if i == 0:
            idle_test += 1
        elif i == 1:
            walking_test += 1
        elif i == 2:
            stairs_test += 1
        elif i == 3:
            running_test += 1

    idle_pred = 0
    walking_pred = 0
    stairs_pred = 0
    running_pred = 0

    for i in y_test_proba:
        if i == 0:
            idle_pred += 1
        elif i == 1:
            walking_pred += 1
        elif i == 2:
            stairs_pred += 1
        elif i == 3:
            running_pred += 1

    print(f'idle test -- {idle_test} pred -- {idle_pred}')
    print(f'walking test -- {walking_test} pred -- {walking_pred}')
    print(f'stairs test -- {stairs_test} pred -- {stairs_pred}')
    print(f'running test -- {running_test} pred -- {running_pred}')

## 3. Класифікація методом опорних векторів SVM (Support Vector Machine)

### 3.1. SVM stat

In [170]:
from sklearn.model_selection import cross_val_score  # оцінювання продуктивності моделі з використанням крос-валідації
from sklearn.metrics import f1_score, accuracy_score  # розрахунок метрик якості моделі
from sklearn import svm

In [171]:
# Завантажимо датасет 'stat_data_Y.pickle'
X, Y = open_data_set('stat')

# Розіб'ємо завантажені дані на навчальну і тестову частини
from sklearn import model_selection
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, Y, test_size=0.3)

### 3.1.1. SVM gamma=0.01

In [172]:
# Створимо класифікатор для багатокласової класифікації з gamma=0.01
classifier = svm.SVC(decision_function_shape='ovo', kernel='rbf', gamma=0.01, probability=True)
# 'svm.SVC' - створення об'єкта класифікатора
# "decision_function_shape='ovo'" - використовується метод "one-vs-one"
# "kernel='rbf'" - задає використання радіально-базової функції (Radial Basis Function, RBF) як ядро SVM
# "gamma=0.01" - параметр, який впливає на форму гіперповерхні рішення
# "probability=True" - вказує, що класифікатор повинен надавати оцінки ймовірності кожного класу при виклику методу predict_proba

In [173]:
# Проведемо навчання класифікатора на навчальній вибірці підготовлених даних но основі часових ознак
classifier.fit(X_train, y_train)

SVC(decision_function_shape='ovo', gamma=0.01, probability=True)

In [174]:
# Оцінимо продуктивність моделі з використанням крос-валідації
scores = cross_val_score(classifier, X_train, y_train, cv=10)  # cv=10 - к-ть фолдів (10-ти кратна крос-валідація)
scores.mean()  # середнє значення продуктивності

0.8489988083377288

In [175]:
# Перевіримо точність передбачень на тестовій вибірці
y_test_proba = classifier.predict_proba(X_test)  # ймовірність приналежності до класів для кожного об'єкта у тестовому наборі
y_test_proba = np.argmax(y_test_proba, axis=1)  # вибір індексу з найвищою ймовірністю (тобто класу з найвищою ймовірністю) для кожного об'єкта у тестовому наборі
accuracy = accuracy_score(y_test, y_test_proba)  # порівнює фактичні мітки (y_test) з передбаченими мітками (y_test_proba) і обчислює точність передбачення
accuracy

0.9788550799381124

In [176]:
make_report(y_test_proba, y_test)

idle test -- 302 pred -- 302
walking test -- 540 pred -- 540
stairs test -- 49 pred -- 49
running test -- 1048 pred -- 1048


### 3.1.2. SVM gamma='scale'

In [177]:
# Створимо класифікатор для багатокласової класифікації з gamma='scale' (параметр за замовчуванням) (атвоматичне масштабування gamma) 
classifier = svm.SVC(decision_function_shape='ovo', probability=True)
# Проведемо навчання класифікатора на навчальній вибірці підготовлених даних но основі часових ознак
classifier.fit(X_train, y_train)

SVC(decision_function_shape='ovo', probability=True)

In [178]:
# Оцінимо продуктивність моделі з використанням крос-валідації
scores = cross_val_score(classifier, X_train, y_train, cv=10)  # cv=10 - к-ть фолдів (10-ти кратна крос-валідація)
scores.mean()

0.975679833557991

In [179]:
# Перевіримо точність передбачень на тестовій вибірці
y_test_proba = classifier.predict_proba(X_test)  # ймовірність приналежності до класів для кожного об'єкта у тестовому наборі
y_test_proba = np.argmax(y_test_proba, axis=1)  # вибір індексу з найвищою ймовірністю (тобто класу з найвищою ймовірністю) для кожного об'єкта у тестовому наборі
accuracy = accuracy_score(y_test, y_test_proba)  # порівнює фактичні мітки (y_test) з передбаченими мітками (y_test_proba) і обчислює точність передбачення
accuracy

0.9855595667870036

In [180]:
make_report(y_test_proba, y_test)

idle test -- 310 pred -- 310
walking test -- 562 pred -- 562
stairs test -- 57 pred -- 57
running test -- 1010 pred -- 1010


### 3.1.3. SVM kernel='linear'

In [181]:
# Створимо класифікатор для багатокласової класифікації з ядром kernel='linear' 
classifier = svm.SVC(decision_function_shape='ovo', probability=True, kernel='linear')
# Проведемо навчання класифікатора на навчальній вибірці підготовлених даних но основі часових ознак
classifier.fit(X_train, y_train)

SVC(decision_function_shape='ovo', kernel='linear', probability=True)

In [182]:
# Оцінимо продуктивність моделі з використанням крос-валідації
scores = cross_val_score(classifier, X_train, y_train, cv=10)  # cv=10 - к-ть фолдів (10-ти кратна крос-валідація)
scores.mean()

0.9955781515559983

In [183]:
# Перевіримо точність передбачень на тестовій вибірці
y_test_proba = classifier.predict_proba(X_test)  # ймовірність приналежності до класів для кожного об'єкта у тестовому наборі
y_test_proba = np.argmax(y_test_proba, axis=1)  # вибір індексу з найвищою ймовірністю (тобто класу з найвищою ймовірністю) для кожного об'єкта у тестовому наборі
accuracy = accuracy_score(y_test, y_test_proba)  # порівнює фактичні мітки (y_test) з передбаченими мітками (y_test_proba) і обчислює точність передбачення
accuracy

0.9927797833935018

In [184]:
make_report(y_test_proba, y_test)

idle test -- 310 pred -- 310
walking test -- 568 pred -- 568
stairs test -- 51 pred -- 51
running test -- 1010 pred -- 1010


### 3.1.4. SVM OVR (One-Vs-Rest)

In [185]:
cls_ovr = svm.SVC(probability=True, kernel='linear')

# Оцінимо продуктивність моделі з використанням крос-валідації
scores = cross_val_score(cls_ovr, X_train, y_train, cv=10)
scores.mean()

0.9955781515559983

In [186]:
# Проведемо навчання класифікатора на навчальній вибірці підготовлених даних но основі часових ознак
cls_ovr.fit(X_train, y_train)

SVC(kernel='linear', probability=True)

In [187]:
# Перевіримо точність передбачень на тестовій вибірці
y_test_proba = cls_ovr.predict_proba(X_test)  # ймовірність приналежності до класів для кожного об'єкта у тестовому наборі
y_test_proba = np.argmax(y_test_proba, axis=1)  # вибір індексу з найвищою ймовірністю (тобто класу з найвищою ймовірністю) для кожного об'єкта у тестовому наборі
accuracy = accuracy_score(y_test, y_test_proba)  # порівнює фактичні мітки (y_test) з передбаченими мітками (y_test_proba) і обчислює точність передбачення
accuracy

0.9927797833935018

In [188]:
make_report(y_test_proba, y_test)

idle test -- 310 pred -- 310
walking test -- 568 pred -- 568
stairs test -- 51 pred -- 51
running test -- 1010 pred -- 1010


### 3.2. SVM fft

In [189]:
from sklearn.model_selection import cross_val_score  # оцінювання продуктивності моделі з використанням крос-валідації
from sklearn.metrics import f1_score, accuracy_score  # розрахунок метрик якості моделі
from sklearn import svm

In [190]:
# Завантажимо датасет 'stat_data_Y.pickle'
X, Y = open_data_set('fft')

# Розіб'ємо завантажені дані на навчальну і тестову частини
from sklearn import model_selection
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, Y, test_size=0.3)

### 3.2.1. SVM gamma=0.01

In [191]:
# Створимо класифікатор для багатокласової класифікації з gamma=0.01
classifier = svm.SVC(decision_function_shape='ovo', kernel='rbf', gamma=0.01, probability=True)
# 'svm.SVC' - створення об'єкта класифікатора
# "decision_function_shape='ovo'" - використовується метод "one-vs-one"
# "kernel='rbf'" - задає використання радіально-базової функції (Radial Basis Function, RBF) як ядро SVM
# "gamma=0.01" - параметр, який впливає на форму гіперповерхні рішення
# "probability=True" - вказує, що класифікатор повинен надавати оцінки ймовірності кожного класу при виклику методу predict_proba

In [192]:
# Проведемо навчання класифікатора на навчальній вибірці підготовлених даних но основі часових ознак
classifier.fit(X_train, y_train)

SVC(decision_function_shape='ovo', gamma=0.01, probability=True)

In [193]:
# Оцінимо продуктивність моделі з використанням крос-валідації
scores = cross_val_score(classifier, X_train, y_train, cv=10)  # cv=10 - к-ть фолдів (10-ти кратна крос-валідація)
scores.mean()  # середнє значення продуктивності

0.9692663462853346

In [194]:
# Перевіримо точність передбачень на тестовій вибірці
y_test_proba = classifier.predict_proba(X_test)  # ймовірність приналежності до класів для кожного об'єкта у тестовому наборі
y_test_proba = np.argmax(y_test_proba, axis=1)  # вибір індексу з найвищою ймовірністю (тобто класу з найвищою ймовірністю) для кожного об'єкта у тестовому наборі
accuracy = accuracy_score(y_test, y_test_proba)  # порівнює фактичні мітки (y_test) з передбаченими мітками (y_test_proba) і обчислює точність передбачення
accuracy

0.9793708096957194

In [195]:
make_report(y_test_proba, y_test)

idle test -- 309 pred -- 309
walking test -- 604 pred -- 604
stairs test -- 31 pred -- 31
running test -- 995 pred -- 995


### 3.2.2. SVM gamma='scale'

In [196]:
# Створимо класифікатор для багатокласової класифікації з gamma='scale' (параметр за замовчуванням) (атвоматичне масштабування gamma) 
classifier = svm.SVC(decision_function_shape='ovo', probability=True)
# Проведемо навчання класифікатора на навчальній вибірці підготовлених даних но основі часових ознак
classifier.fit(X_train, y_train)

SVC(decision_function_shape='ovo', probability=True)

In [197]:
# Оцінимо продуктивність моделі з використанням крос-валідації
scores = cross_val_score(classifier, X_train, y_train, cv=10)  # cv=10 - к-ть фолдів (10-ти кратна крос-валідація)
scores.mean()

0.9812068999199047

In [198]:
# Перевіримо точність передбачень на тестовій вибірці
y_test_proba = classifier.predict_proba(X_test)  # ймовірність приналежності до класів для кожного об'єкта у тестовому наборі
y_test_proba = np.argmax(y_test_proba, axis=1)  # вибір індексу з найвищою ймовірністю (тобто класу з найвищою ймовірністю) для кожного об'єкта у тестовому наборі
accuracy = accuracy_score(y_test, y_test_proba)  # порівнює фактичні мітки (y_test) з передбаченими мітками (y_test_proba) і обчислює точність передбачення
accuracy

0.9824651882413615

In [199]:
make_report(y_test_proba, y_test)

idle test -- 306 pred -- 306
walking test -- 605 pred -- 605
stairs test -- 30 pred -- 30
running test -- 998 pred -- 998


### 3.2.3. SVM kernel='linear'

In [200]:
# Створимо класифікатор для багатокласової класифікації з ядром kernel='linear' 
classifier = svm.SVC(decision_function_shape='ovo', probability=True, kernel='linear')
# Проведемо навчання класифікатора на навчальній вибірці підготовлених даних но основі часових ознак
classifier.fit(X_train, y_train)

SVC(decision_function_shape='ovo', kernel='linear', probability=True)

In [201]:
# Оцінимо продуктивність моделі з використанням крос-валідації
scores = cross_val_score(classifier, X_train, y_train, cv=10)  # cv=10 - к-ть фолдів (10-ти кратна крос-валідація)
scores.mean()

0.9759025376545744

In [202]:
# Перевіримо точність передбачень на тестовій вибірці
y_test_proba = classifier.predict_proba(X_test)  # ймовірність приналежності до класів для кожного об'єкта у тестовому наборі
y_test_proba = np.argmax(y_test_proba, axis=1)  # вибір індексу з найвищою ймовірністю (тобто класу з найвищою ймовірністю) для кожного об'єкта у тестовому наборі
accuracy = accuracy_score(y_test, y_test_proba)  # порівнює фактичні мітки (y_test) з передбаченими мітками (y_test_proba) і обчислює точність передбачення
accuracy

0.9824651882413615

In [203]:
make_report(y_test_proba, y_test)

idle test -- 308 pred -- 308
walking test -- 613 pred -- 613
stairs test -- 22 pred -- 22
running test -- 996 pred -- 996


### 3.2.4. SVM OVR (One-Vs-Rest)

In [204]:
cls_ovr = svm.SVC(probability=True, kernel='linear')

# Оцінимо продуктивність моделі з використанням крос-валідації
scores = cross_val_score(cls_ovr, X_train, y_train, cv=10)
scores.mean()

0.9759025376545744

In [205]:
# Проведемо навчання класифікатора на навчальній вибірці підготовлених даних но основі часових ознак
cls_ovr.fit(X_train, y_train)

SVC(kernel='linear', probability=True)

In [206]:
# Перевіримо точність передбачень на тестовій вибірці
y_test_proba = cls_ovr.predict_proba(X_test)  # ймовірність приналежності до класів для кожного об'єкта у тестовому наборі
y_test_proba = np.argmax(y_test_proba, axis=1)  # вибір індексу з найвищою ймовірністю (тобто класу з найвищою ймовірністю) для кожного об'єкта у тестовому наборі
accuracy = accuracy_score(y_test, y_test_proba)  # порівнює фактичні мітки (y_test) з передбаченими мітками (y_test_proba) і обчислює точність передбачення
accuracy

0.9824651882413615

In [207]:
make_report(y_test_proba, y_test)

idle test -- 308 pred -- 308
walking test -- 615 pred -- 615
stairs test -- 20 pred -- 20
running test -- 996 pred -- 996


### 3.3. SVM stat+fft

In [208]:
from sklearn.model_selection import cross_val_score  # оцінювання продуктивності моделі з використанням крос-валідації
from sklearn.metrics import f1_score, accuracy_score  # розрахунок метрик якості моделі
from sklearn import svm

In [209]:
# Завантажимо датасет 'stat_data_Y.pickle'
X, Y = open_data_set('fft_stat')

# Розіб'ємо завантажені дані на навчальну і тестову частини
from sklearn import model_selection
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, Y, test_size=0.3)

### 3.3.1. SVM gamma=0.01

In [210]:
# Створимо класифікатор для багатокласової класифікації з gamma=0.01
classifier = svm.SVC(decision_function_shape='ovo', kernel='rbf', gamma=0.01, probability=True)
# 'svm.SVC' - створення об'єкта класифікатора
# "decision_function_shape='ovo'" - використовується метод "one-vs-one"
# "kernel='rbf'" - задає використання радіально-базової функції (Radial Basis Function, RBF) як ядро SVM
# "gamma=0.01" - параметр, який впливає на форму гіперповерхні рішення
# "probability=True" - вказує, що класифікатор повинен надавати оцінки ймовірності кожного класу при виклику методу predict_proba

In [211]:
# Проведемо навчання класифікатора на навчальній вибірці підготовлених даних но основі часових ознак
classifier.fit(X_train, y_train)

SVC(decision_function_shape='ovo', gamma=0.01, probability=True)

In [212]:
# Оцінимо продуктивність моделі з використанням крос-валідації
scores = cross_val_score(classifier, X_train, y_train, cv=10)  # cv=10 - к-ть фолдів (10-ти кратна крос-валідація)
scores.mean()  # середнє значення продуктивності

0.845452636308582

In [213]:
# Перевіримо точність передбачень на тестовій вибірці
y_test_proba = classifier.predict_proba(X_test)  # ймовірність приналежності до класів для кожного об'єкта у тестовому наборі
y_test_proba = np.argmax(y_test_proba, axis=1)  # вибір індексу з найвищою ймовірністю (тобто класу з найвищою ймовірністю) для кожного об'єкта у тестовому наборі
accuracy = accuracy_score(y_test, y_test_proba)  # порівнює фактичні мітки (y_test) з передбаченими мітками (y_test_proba) і обчислює точність передбачення
accuracy

0.9731820526044352

In [214]:
make_report(y_test_proba, y_test)

idle test -- 302 pred -- 302
walking test -- 508 pred -- 508
stairs test -- 42 pred -- 42
running test -- 1087 pred -- 1087


### 3.3.2. SVM gamma='scale'

In [215]:
# Створимо класифікатор для багатокласової класифікації з gamma='scale' (параметр за замовчуванням) (атвоматичне масштабування gamma) 
classifier = svm.SVC(decision_function_shape='ovo', probability=True)
# Проведемо навчання класифікатора на навчальній вибірці підготовлених даних но основі часових ознак
classifier.fit(X_train, y_train)

SVC(decision_function_shape='ovo', probability=True)

In [216]:
# Оцінимо продуктивність моделі з використанням крос-валідації
scores = cross_val_score(classifier, X_train, y_train, cv=10)  # cv=10 - к-ть фолдів (10-ти кратна крос-валідація)
scores.mean()

0.9734689093359901

In [217]:
# Перевіримо точність передбачень на тестовій вибірці
y_test_proba = classifier.predict_proba(X_test)  # ймовірність приналежності до класів для кожного об'єкта у тестовому наборі
y_test_proba = np.argmax(y_test_proba, axis=1)  # вибір індексу з найвищою ймовірністю (тобто класу з найвищою ймовірністю) для кожного об'єкта у тестовому наборі
accuracy = accuracy_score(y_test, y_test_proba)  # порівнює фактичні мітки (y_test) з передбаченими мітками (y_test_proba) і обчислює точність передбачення
accuracy

0.9871067560598247

In [218]:
make_report(y_test_proba, y_test)

idle test -- 318 pred -- 318
walking test -- 534 pred -- 534
stairs test -- 50 pred -- 50
running test -- 1037 pred -- 1037


### 3.3.3. SVM kernel='linear'

In [219]:
# Створимо класифікатор для багатокласової класифікації з ядром kernel='linear' 
classifier = svm.SVC(decision_function_shape='ovo', probability=True, kernel='linear')
# Проведемо навчання класифікатора на навчальній вибірці підготовлених даних но основі часових ознак
classifier.fit(X_train, y_train)

SVC(decision_function_shape='ovo', kernel='linear', probability=True)

In [220]:
# Оцінимо продуктивність моделі з використанням крос-валідації
scores = cross_val_score(classifier, X_train, y_train, cv=10)  # cv=10 - к-ть фолдів (10-ти кратна крос-валідація)
scores.mean()

0.9927044872921916

In [221]:
# Перевіримо точність передбачень на тестовій вибірці
y_test_proba = classifier.predict_proba(X_test)  # ймовірність приналежності до класів для кожного об'єкта у тестовому наборі
y_test_proba = np.argmax(y_test_proba, axis=1)  # вибір індексу з найвищою ймовірністю (тобто класу з найвищою ймовірністю) для кожного об'єкта у тестовому наборі
accuracy = accuracy_score(y_test, y_test_proba)  # порівнює фактичні мітки (y_test) з передбаченими мітками (y_test_proba) і обчислює точність передбачення
accuracy

0.9963898916967509

In [222]:
make_report(y_test_proba, y_test)

idle test -- 318 pred -- 318
walking test -- 536 pred -- 536
stairs test -- 48 pred -- 48
running test -- 1037 pred -- 1037


### 3.3.4. SVM OVR (One-Vs-Rest)

In [223]:
cls_ovr = svm.SVC(probability=True, kernel='linear')

# Оцінимо продуктивність моделі з використанням крос-валідації
scores = cross_val_score(cls_ovr, X_train, y_train, cv=10)
scores.mean()

0.9927044872921916

In [224]:
# Проведемо навчання класифікатора
cls_ovr.fit(X_train, y_train)

SVC(kernel='linear', probability=True)

In [225]:
# Перевіримо точність передбачень на тестовій вибірці
y_test_proba = cls_ovr.predict_proba(X_test)  # ймовірність приналежності до класів для кожного об'єкта у тестовому наборі
y_test_proba = np.argmax(y_test_proba, axis=1)  # вибір індексу з найвищою ймовірністю (тобто класу з найвищою ймовірністю) для кожного об'єкта у тестовому наборі
accuracy = accuracy_score(y_test, y_test_proba)  # порівнює фактичні мітки (y_test) з передбаченими мітками (y_test_proba) і обчислює точність передбачення
accuracy

0.9963898916967509

In [226]:
make_report(y_test_proba, y_test)

idle test -- 318 pred -- 318
walking test -- 536 pred -- 536
stairs test -- 48 pred -- 48
running test -- 1037 pred -- 1037


## 4. Класифікація методом випадкового лісу (Random Forest)

### 4.1. RF stat

In [227]:
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier

In [228]:
# Створимо екземпляр RF
clf = RandomForestClassifier(n_estimators=15, max_depth=None)
# 'n_estimators=15' - кількість "дерев в лісі";
# 'max_depth=None' - цей параметр визначає максимальну глибину кожного дерева. None означає, що дерева не обмежені по глибині, 
# і вони будуть будуватися до тих пір, поки кожен лист не міститиме менше, ніж min_samples_split елементів

In [229]:
# Завантажимо датасет 'stat_data_Y.pickle'
X, Y = open_data_set('stat')

# Розіб'ємо завантажені дані на навчальну і тестову частини
from sklearn import model_selection
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, Y, test_size=0.3)

In [230]:
# Навчаємо модель
clf = clf.fit(X_train, y_train)

In [231]:
# Оцінимо продуктивність моделі з використанням крос-валідації
scores = cross_val_score(clf, X_train, y_train, cv=10)  # cv=10 - к-ть фолдів (10-ти кратна крос-валідація)
scores.mean()  # середнє значення продуктивності

0.9980098263298756

In [232]:
# Оцінимо якість навчання на тестовій вибірці
y_pred = clf.predict(X_test)

In [233]:
# Порахуємо точність моделі
accuracy = accuracy_score(y_test, y_pred)
accuracy

0.9953584321815369

In [234]:
from sklearn.metrics import f1_score  # метрика продуктивності моделі
f1score = f1_score(y_test, y_pred, average='macro')  # макро-усереднення розрахунку f1-міри
f1score

0.9769320714110201

In [235]:
# Створимо ф-ю для підрахунку кількості індексів класів
def make_report_forest(y_pred, y_test):

    idle_test = 0
    walking_test = 0
    stairs_test = 0
    running_test = 0

    for i in y_test:
        if i == 0:
            idle_test += 1
        elif i == 1:
            walking_test += 1
        elif i == 2:
            stairs_test += 1
        elif i == 3:
            running_test += 1

    idle_pred = 0
    walking_pred = 0
    stairs_pred = 0
    running_pred = 0

    for i in y_pred:
        if i == 0:
            idle_pred += 1
        elif i == 1:
            walking_pred += 1
        elif i == 2:
            stairs_pred += 1
        elif i == 3:
            running_pred += 1

    print(f'idle test -- {idle_test} pred -- {idle_pred}')
    print(f'walking test -- {walking_test} pred -- {walking_pred}')
    print(f'stairs test -- {stairs_test} pred -- {stairs_pred}')
    print(f'running test -- {running_test} pred -- {running_pred}')

In [236]:
make_report_forest(y_pred, y_test)

idle test -- 319 pred -- 319
walking test -- 547 pred -- 556
stairs test -- 58 pred -- 49
running test -- 1015 pred -- 1015


### 4.2. RF fft

In [237]:
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier

In [238]:
# Створимо екземпляр RF
clf = RandomForestClassifier(n_estimators=15, max_depth=None)
# 'n_estimators=15' - кількість "дерев в лісі";
# 'max_depth=None' - цей параметр визначає максимальну глибину кожного дерева. None означає, що дерева не обмежені по глибині, 
# і вони будуть будуватися до тих пір, поки кожен лист не міститиме менше, ніж min_samples_split елементів

In [239]:
# Завантажимо датасет 'stat_data_Y.pickle'
X, Y = open_data_set('fft')

# Розіб'ємо завантажені дані на навчальну і тестову частини
from sklearn import model_selection
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, Y, test_size=0.3)

In [240]:
# Навчаємо модель
clf = clf.fit(X_train, y_train)

In [241]:
# Оцінимо продуктивність моделі з використанням крос-валідації
scores = cross_val_score(clf, X_train, y_train, cv=10)  # cv=10 - к-ть фолдів (10-ти кратна крос-валідація)
scores.mean()  # середнє значення продуктивності

0.9812078766922582

In [242]:
# Оцінимо якість навчання на тестовій вибірці
y_pred = clf.predict(X_test)

In [243]:
# Порахуємо точність моделі
accuracy = accuracy_score(y_test, y_pred)
accuracy

0.9716348633316142

In [244]:
from sklearn.metrics import f1_score  # метрика продуктивності моделі
f1score = f1_score(y_test, y_pred, average='macro')  # макро-усереднення розрахунку f1-міри
f1score

0.8014993003085495

In [245]:
make_report_forest(y_pred, y_test)

idle test -- 302 pred -- 300
walking test -- 546 pred -- 597
stairs test -- 61 pred -- 9
running test -- 1030 pred -- 1033


### 4.3. RF stat+fft

In [246]:
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier

In [247]:
# Створимо екземпляр RF
clf = RandomForestClassifier(n_estimators=15, max_depth=None)
# 'n_estimators=15' - кількість "дерев в лісі";
# 'max_depth=None' - цей параметр визначає максимальну глибину кожного дерева. None означає, що дерева не обмежені по глибині, 
# і вони будуть будуватися до тих пір, поки кожен лист не міститиме менше, ніж min_samples_split елементів

In [248]:
# Завантажимо датасет 'stat_data_Y.pickle'
X, Y = open_data_set('fft_stat')

# Розіб'ємо завантажені дані на навчальну і тестову частини
from sklearn import model_selection
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, Y, test_size=0.3)

In [249]:
# Навчаємо модель
clf = clf.fit(X_train, y_train)

In [250]:
# Оцінимо продуктивність моделі з використанням крос-валідації
scores = cross_val_score(clf, X_train, y_train, cv=10)  # cv=10 - к-ть фолдів (10-ти кратна крос-валідація)
scores.mean()  # середнє значення продуктивності

0.9953569126179452

In [251]:
# Оцінимо якість навчання на тестовій вибірці
y_pred = clf.predict(X_test)

In [252]:
# Порахуємо точність моделі
accuracy = accuracy_score(y_test, y_pred)
accuracy

0.9953584321815369

In [253]:
from sklearn.metrics import f1_score  # метрика продуктивності моделі
f1score = f1_score(y_test, y_pred, average='macro')  # макро-усереднення розрахунку f1-міри
f1score

0.9727471466976534

In [254]:
make_report_forest(y_pred, y_test)

idle test -- 311 pred -- 311
walking test -- 566 pred -- 575
stairs test -- 49 pred -- 40
running test -- 1013 pred -- 1013
